# Weather Visualization for Portland, OR: 1940 - 2020

- Notebook Author: [Trenton McKinney][1]
- Just for fun

  [1]: https://trenton3983.github.io/

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import numpy as np
import calendar
from math import floor
from datetime import date

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 10.0)
# plt.style.use('ggplot')
# sns.set_style("white")

# Load and Clean Data

## Load a local copy

In [ ]:
p = Path().cwd().parents[0] / 'data/Portland_dailyclimatedata1940-2019.xlsx'

In [ ]:
pdx_19 = pd.read_excel(p, sheet_name='Portland_dailyclimatedata1940-2', skiprows=list(range(6)))

## Load from website

- Data from [NOAA Nation Weather Service Forecast Office: Portland, OR][1]

  [1]: https://w2.weather.gov/climate/local_data.php?wfo=pqr

In [ ]:
url = 'http://www.weather.gov/source/pqr/climate/webdata/Portland_dailyclimatedata.csv'
pdx_19 = pd.read_csv(url, skiprows=list(range(6)))

## View initial dataframe

In [ ]:
pdx_19

## Transform and clean data

In [ ]:
# Drop unused column
pdx_19.drop(columns=['AVG or Total'], inplace=True)

In [ ]:
# add prefix to days for use with wide_to_long
pdx_19.columns = list(pdx_19.columns[:3]) + [f'v_{day}' for day in pdx_19.columns[3:]]

In [ ]:
# Select TX (max temp) and TN (min temp)
pdx_19.rename(columns={'Unnamed: 2': 'TYPE'}, inplace=True)
pdx_19 = pdx_19[pdx_19.TYPE.isin(['TX', 'TN', 'PR'])]

### Convert to Tidy format

In [ ]:
# reshape the data to tidy
pdx = pd.wide_to_long(pdx_19, stubnames='v', sep='_', i=['YR', 'MO', 'TYPE'], j='day').reset_index()
pdx

In [ ]:
# Give a more descriptive name
pdx.TYPE = pdx.TYPE.map({'TX': 'MAX', 'TN': 'MIN', 'PR': 'PRE'})

In [ ]:
# rename so they can be used to create a date column
pdx.rename(columns={'YR': 'year', 'MO': 'month'}, inplace=True)

In [ ]:
# - denotes days that don't exist for a given month; drop those
pdx = pdx[pdx.v != '-'].copy()

In [ ]:
# create date column
pdx['date'] = pd.to_datetime(pdx[['year', 'month', 'day']])

In [ ]:
# add decade
pdx['dec'] = pdx.year.apply(lambda x: f"{floor(x/10)*10}'s")

In [ ]:
pdx.v.replace({'M': np.nan, 'T': np.nan}, inplace=True)

In [ ]:
# convert from str to float
pdx.v = pdx.v.astype('float')

In [ ]:
# add bins
pdx['range'] = pd.cut(pdx.v, bins=[0, 64, 74, 84, 94, 200], labels=['< 65', '65 - 74', '75 - 84', '85 - 94', '>= 95'])

In [ ]:
display(pdx.head())
display(pdx.tail())

# Create max temperature dataframe

In [ ]:
pdx_max = pdx[pdx.TYPE == 'MAX'].reset_index(drop=True)
pdx_max

# Create January to May (inclusive) dataframe

In [ ]:
pdx_max_jan_may = pdx_max[(pdx_max.date.dt.month >= 1) & (pdx_max.date.dt.month < 6)]

## Groupby `year` and `range`

In [ ]:
pdx_max_g = pdx_max_jan_may.groupby([pdx_max_jan_may.date.dt.year, 'range'])['v'].agg('count').reset_index(level=0)

In [ ]:
display(pdx_max_g.head())
display(pdx_max_g.tail())

## Plot the `groupby` dataframe

In [ ]:
years = list(range(1950, 2021, 10))

with sns.axes_style("darkgrid"):
    for year in years:
        plt.figure()
        data = pdx_max_g[(pdx_max_g.date >= year - 10) & (pdx_max_g.date < year)]
        ax = sns.barplot(x=data.index, y=data.v, hue=data.date)
        
        plt.annotate('Prepared By: Trenton McKinney', xy=(3, 140.1), xytext=(3, 140.1), fontsize=8)
        
        for p in ax.patches:
            if p.get_height() > 0:
                ax.annotate(format(p.get_height(), '.0f'),
                            (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha = 'center', va = 'center', fontsize=8,
                            xytext = (0, 10), textcoords = 'offset points')
        
        plt.ylim(0, 150)
        plt.ylabel('Days')
        plt.xlabel('High Temperatures °F')
        plt.title(f"Portland, OR\nJan - May High Temperature Days: {year-10}'s")

# Create June dataframe

In [ ]:
pdx_max_june = pdx_max[(pdx_max.date.dt.month == 6)]
pdx_max_june = pdx_max_june.groupby([pdx_max_june.date.dt.year, 'range'])['v'].agg('count').reset_index(level=0)
pdx_max_june

## Plot June

In [ ]:
years = list(range(1950, 2021, 10))

with sns.axes_style("darkgrid"):
    for year in years:
        plt.figure()
        data = pdx_max_june[(pdx_max_june.date >= year - 10) & (pdx_max_june.date < year)]
        ax = sns.barplot(x=data.index, y=data.v, hue=data.date)
        
        plt.annotate('Prepared By: Trenton McKinney', xy=(0, 24.3), xytext=(0, 24.3), fontsize=8)
        
        for p in ax.patches:
            if p.get_height() > 0:
                ax.annotate(format(p.get_height(), '.0f'),
                            (p.get_x() + p.get_width() / 2., p.get_height()),
                            ha='center', va='center', fontsize=9,
                            xytext=(0, 10), textcoords='offset points')
        
        plt.ylim(0, 25)
        plt.ylabel('Days')
        plt.xlabel('High Temperatures °F')
        plt.title(f"Portland, OR\nJune High Temperature Days: {year-10}'s")

# Resample max monthly mean

In [ ]:
pdx_m_mean = pdx_max.set_index('date').resample('m').mean()

In [ ]:
years = list(range(1950, 2021, 10))

with sns.axes_style("darkgrid"):
    for year in years:
        data = pdx_m_mean[(pdx_m_mean.index.year >= year - 10) & (pdx_m_mean.index.year < year)]
        ax = sns.lineplot(x=data.index, y=data.v)
    
    plt.annotate('Prepared By: Trenton McKinney', xy=(730120, 32), xytext=(730120, 32), fontsize=10)
    plt.xlabel('Year')
    plt.ylabel('Monthly Mean Max Temperature °F')
    plt.title(f"Portland, OR\nResampled Monthly Mean High Temperature")

# Mean: Resample max yearly

In [ ]:
pdx_y_mean = pdx_max[pdx_max.year > 1940].set_index('date').resample('y').mean()
# filtered out 1940 because it's a partial year (only 2 months)

In [ ]:
years = list(range(1950, 2021, 10))

with sns.axes_style("darkgrid"):
    for year in years:
        data = pdx_y_mean[(pdx_y_mean.index.year >= year - 10) & (pdx_y_mean.index.year < year)]
        ax = sns.lineplot(x=data.index, y=data.v)

    plt.annotate('Prepared By: Trenton McKinney', xy=(730120, 58.7), xytext=(730120, 58.7), fontsize=9)
    plt.xlabel('Year')
    plt.ylabel('Yearly Mean Max Temperature °F')
    plt.title(f"Portland, OR\nResampled Yearly Mean High Temperature")

# Mean: Groupby year & month

In [ ]:
mym = pdx_max.groupby([pdx_max.date.dt.year, pdx_max.date.dt.month])['v'].mean().unstack(level=0)
mym = mym.iloc[:, :-1]  # there's no data in 2020 so drop that column
mym.iloc[:, :5]

In [ ]:
ix = list(range(0, len(mym.columns), 10))

with sns.axes_style("darkgrid"):
    for i in ix:
        plt.figure()
        data = mym.iloc[:, i:i+10]
        sns.lineplot(data=data, markers=None, dashes=False)
        plt.annotate('Prepared By: Trenton McKinney', xy=(10, 31), xytext=(10, 31), fontsize=10)

        plt.xticks(np.arange(1, 13), calendar.month_name[1:13])
        plt.ylim(30, 100)
        plt.xlabel('Month')
        plt.ylabel('Mean Max Temperature °F')
        plt.title(f"Portland, OR\nMonthly Mean per Year High Temperature")
        plt.show()

# Mean: Groupby decade & month

In [ ]:
mymd = pdx_max.groupby([pdx_max.dec, pdx_max.date.dt.month])['v'].mean().unstack(level=0)
mymd = mymd.iloc[:, :-1]  # No data for 2020's
mymd

In [ ]:
with sns.axes_style("darkgrid"):

    plt.figure()
    sns.lineplot(data=mymd, markers=None, dashes=False)
    plt.annotate('Prepared By: Trenton McKinney', xy=(10, 40.5), xytext=(10, 40.5), fontsize=10)
    plt.xticks(np.arange(1, 13), calendar.month_name[1:13])
    plt.ylim(40, 85)
    plt.xlabel('Month')
    plt.ylabel('Mean Max Temperature °F')
    plt.title(f"Portland, OR\nMonthly Mean Per Decade High Temperature")
    plt.show()

# 2020 Data

In [ ]:
p = Path().cwd().parents[0] / 'data/Portland_dailyclimatedata1940-2019.xlsx'
pdx_20 = pd.read_excel(p, sheet_name='2020')

## Transform and clean data

In [ ]:
# drop not used columns
pdx_20.drop(columns=['Departure', 'HDD', 'CDD', 'New_Snow', 'Snow_Depth'], inplace=True)

In [ ]:
# change name to facilitate wide_to_long
pdx_20.rename(columns={'Date': 'date', 'MAX': 'v1', 'MIN': 'v2', 'AVG': 'v3', 'Precip': 'v4'}, inplace=True)

In [ ]:
# add bins
pdx_20['w1'] = pd.cut(pdx_20.v1, bins=[0, 64, 74, 84, 94, 200], labels=['< 65', '65 - 74', '75 - 84', '85 - 94', '>= 95'])

In [ ]:
pdx_20

### Convert to Tidy format

In [ ]:
pdx_20_l = pd.wide_to_long(pdx_20, stubnames=['v', 'w'], i=['date'], j='TYPE').reset_index()
pdx_20_l

In [ ]:
pdx_20_l.TYPE = pdx_20_l.TYPE.map({1: 'MAX', 2: 'MIN', 3: 'AVG', 4: 'PRE'})

In [ ]:
pdx_20_l['year'] = pdx_20_l.date.dt.year
pdx_20_l['month'] = pdx_20_l.date.dt.month
pdx_20_l['day'] = pdx_20_l.date.dt.day

In [ ]:
pdx_20_l.rename(columns={'w': 'range'}, inplace=True)

In [ ]:
pdx_20_l.v.replace({'M': np.nan, 'T': np.nan}, inplace=True)

In [ ]:
# add decade
pdx_20_l['dec'] = pdx_20_l.year.apply(lambda x: f"{floor(x/10)*10}'s")

In [ ]:
# MAX temp data only
pdx_20_lm = pdx_20_l[pdx_20_l.TYPE == 'MAX']
pdx_20_lm

In [ ]:
# groupby year & range, then count
pdx_20_lmg = pdx_20_lm.groupby([pdx_20_lm.date.dt.year, 'range'])['v'].agg('count').sort_values(ascending=False).reset_index(level=0)

In [ ]:
display(pdx_20_lmg.head())
display(pdx_20_lmg.tail())

## Plot 2020 & 2019 High Temperature Days

In [ ]:
pdx_max_g_2019 = pdx_max_g[pdx_max_g.date == 2019]
data = pdx_max_g_2019.append(pdx_20_lmg)

with sns.axes_style("darkgrid"):
    ax = sns.barplot(x=data.index, y=data.v, hue=data.date)

    plt.annotate('Prepared By: Trenton McKinney', xy=(3, 140.1), xytext=(3, 140.1), fontsize=8)

    for p in ax.patches:
        if p.get_height() > 0:
            ax.annotate(format(p.get_height(), '.0f'),
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha = 'center', va = 'center', fontsize=8,
                        xytext = (0, 10), textcoords = 'offset points')
            
    plt.ylim(0, 150)
    plt.ylabel('Days')
    plt.xlabel('High Temperatures °F')
    plt.title(f"Portland, OR\n 2020: Jan - May 24\n2019: Jan - May (all)\nHigh Temperature Days")

# Precipitation

## 2020

In [ ]:
# PRE: precipitation data only
pdx_20_lp = pdx_20_l[(pdx_20_l.TYPE == 'PRE')]
pdx_20_lp.head()

In [ ]:
plt.figure(figsize=(25, 10))
with sns.axes_style("darkgrid"):

    sns.barplot(pdx_20_lp.date, pdx_20_lp.v)

    plt.annotate('Prepared By: Trenton McKinney', xy=(120, 0.9), xytext=(120, 0.9), fontsize=8)
    ticks, labels = plt.xticks(rotation=90)
    labels = [label.get_text()[:10] for label in labels]
    plt.xticks(ticks=ticks, labels=labels)
    
    plt.ylabel('Precipitation (inches)')
    plt.xlabel('Date')
    plt.title(f"Portland, OR\n 2020: Jan - May 24\nPrecipitation Days")

In [ ]:
with sns.axes_style("darkgrid"):

    sns.boxplot(pdx_20_lp.date.dt.month, pdx_20_lp.v)
    
    plt.xticks(np.arange(0, 5), calendar.month_name[1:6])

    plt.annotate('Prepared By: Trenton McKinney', xy=(3, 0.9), xytext=(3, 0.9), fontsize=8)
    
    plt.ylabel('Precipitation (inches)')
    plt.xlabel('Month')
    plt.title(f"Portland, OR\n 2020: Jan - May 24\nPrecipitation")

## 1940 - 2020

In [ ]:
pdx_precip = pdx[(pdx.TYPE == 'PRE')].append(pdx_20_lp).reset_index(drop=True)

In [ ]:
pdx_precip.drop(columns=['range'], inplace=True)

In [ ]:
pdx_precip

### Mean Monthly Precipitation per Decade: 2940 - 2020

In [ ]:
pmymd = pdx_precip.groupby([pdx_precip.dec, pdx_precip.date.dt.month])['v'].mean().unstack(level=0)
pmymd

In [ ]:
with sns.axes_style("darkgrid"):

    plt.figure()
    sns.lineplot(data=pmymd, markers=None, dashes=False)
    plt.annotate('Prepared By: Trenton McKinney', xy=(10, 0.06), xytext=(10, 0.06), fontsize=10)
    plt.xticks(np.arange(1, 13), calendar.month_name[1:13])

    plt.xlabel('Month')
    plt.ylabel('Mean Precipitation (inches)')
    plt.title(f"Portland, OR\nMonthly Mean Amount of Precipitation by Decade")
    plt.show()

In [ ]:
pmymds = pmymd.stack().reset_index()

with sns.axes_style("darkgrid"):

    plt.figure(figsize=(20, 10))
    ax = sns.barplot('date', 0, data=pmymds, hue='dec')

    for p in ax.patches:
        if p.get_height() > 0:
            ax.annotate(format(p.get_height(), '.2f'),
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', fontsize=9, rotation=90,
                        xytext=(0, 10), textcoords='offset points')
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.annotate('Prepared By: Trenton McKinney', xy=(9, 0.351), xytext=(9, 0.351), fontsize=10)
    plt.xticks(np.arange(0, 12), calendar.month_name[1:13])
    plt.xlabel('Month')
    plt.ylabel('Mean Precipitation (inches)')
    plt.title(f"Portland, OR\nMonthly Mean Amount of Precipitation by Decade")
    plt.show()

- **Note the 2020's are only made up of 4.5 months of data**
- Other decades are the mean m over 10 years.

### Total Precipitation Days: 1940 - 2020

In [ ]:
precip_days_year = pdx_precip[pdx_precip.v > 0].groupby('year', as_index=False)['year'].agg({'count': 'count'})

with sns.axes_style("darkgrid"):
    ax = sns.barplot(x='year', y='count', data=precip_days_year)
    plt.xticks(rotation=90)

    # plt.annotate('Prepared By: Trenton McKinney', xy=(3, 140.1), xytext=(3, 140.1), fontsize=8)

    for p in ax.patches:
        if p.get_height() > 0:
            ax.annotate(format(p.get_height(), '.0f'),
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha = 'center', va = 'center', fontsize=9, rotation=90,
                        xytext = (0, 10), textcoords = 'offset points')

    plt.ylabel('Days of Precipitation')
    plt.xlabel('Year')
    plt.title(f"Portland, OR\nPrecipitation Days Per Year")

    plt.show()

- **1940 and 2020 do not have complete data for the year**

### Mean Monthly Precipitation: 1940 - 2020

In [ ]:
precip_days_month = pdx_precip.groupby('month', as_index=False)['v'].agg({'mean': 'mean'})
ax = sns.barplot(x='month', y='mean', data=precip_days_month)
plt.xticks(rotation=90)

# plt.annotate('Prepared By: Trenton McKinney', xy=(3, 140.1), xytext=(3, 140.1), fontsize=8)
plt.xticks(np.arange(0, 12), calendar.month_name[1:13])

for p in ax.patches:
    if p.get_height() > 0:
        ax.annotate(format(p.get_height(), '.2f'),
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center', fontsize=8,
                    xytext = (0, 10), textcoords = 'offset points')

plt.ylabel('Mean Precipitation (inches)')
plt.xlabel('Month')
plt.title(f"Portland, OR\nMean Precipitation Per Month: 1940 - 2020")

plt.show()

### Total Jan - May Precipitation: 2018 - 2020

In [ ]:
gb_18_20 = pdx_precip[(pdx_precip.year.isin([2018, 2019, 2020])) & (pdx_precip.month < 6)].groupby(['year', 'month']).agg({'v': sum}).reset_index()

In [ ]:
ax = sns.barplot(x=gb_18_20.month, y=gb_18_20.v, hue=gb_18_20.year)
plt.xticks(np.arange(0, 5), calendar.month_name[1:6])

for p in ax.patches:
    if p.get_height() > 0:
        ax.annotate(format(p.get_height(), '.2f'),
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha = 'center', va = 'center', fontsize=8,
                    xytext = (0, 10), textcoords = 'offset points')

plt.ylabel('Total Precipitation (inches)')
plt.xlabel('Month')
plt.title(f"Portland, OR\nTotal Monthly Precipitation\n(Jan - May): 2018 - 2020")
plt.show()

### Monthly Total Precipitation

In [ ]:
years = list(range(1950, 2031, 10))

with sns.axes_style("darkgrid"):
    for year in years:
        plt.figure()
        data = pdx_precip[(pdx_precip.year >= year - 10) & (pdx_precip.year < year)]
        data = data.groupby(['year', 'month']).agg({'v': sum}).reset_index()
        ax = sns.barplot(x=data.month, y=data.v, hue=data.year)
        
#         plt.annotate('Prepared By: Trenton McKinney', xy=(3, 140.1), xytext=(3, 140.1), fontsize=8)
        
        for p in ax.patches:
            if p.get_height() > 0:
                ax.annotate(format(p.get_height(), '.2f'),
                            (p.get_x() + p.get_width() / 2., p.get_height() + 0.2),
                            ha = 'center', va = 'center', fontsize=9, rotation=90,
                            xytext = (0, 10), textcoords = 'offset points')

        plt.xticks(np.arange(0, 12), calendar.month_name[1:13])
        plt.ylim(0, 16)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.ylabel('Total Rain (inches)')
        plt.xlabel('Month')
        plt.title(f"Portland, OR\nTotal Rain Per Month: {year-10}'s")

## 2011 - 2020

In [ ]:
pdx_precip_11_00 = pdx_precip[(pdx_precip.year > 2010)]
pdx_precip_11_00

In [ ]:
with sns.axes_style("darkgrid"):

    sns.boxplot(pdx_precip_11_00.date.dt.year, pdx_precip_11_00.v, color='lightgray')
    sns.swarmplot(pdx_precip_11_00.date.dt.year, pdx_precip_11_00.v, size=2.5)
    
    plt.annotate('Prepared By: Trenton McKinney', xy=(7, 2.51), xytext=(7, 2.51), fontsize=8)
    
    plt.ylabel('Precipitation (inches)')
    plt.xlabel('Year')
    plt.title(f"Portland, OR\n 2011 - 2020 Amount of Precipitation")